In [6]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [7]:
# Verileri yükle
labels = pd.read_csv("dog-breed-identification/labels.csv")

# Dosya adlarına uzantı ekleyerek yeni bir sütun oluştur
labels['file_path'] = labels['id'] + '.jpg'

# Veri artırma işlemi
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Veri setini eğitim ve test olarak böl
train_data, valid_data = train_test_split(labels, test_size=0.2, random_state=42)

# Veri setindeki sınıf sayısını al
num_classes = len(labels['breed'].unique())

In [8]:
# train_generator ve valid_generator'daki class_mode 'sparse' olarak değiştirildi
train_generator = datagen.flow_from_dataframe(
    train_data,
    directory="dog-breed-identification/train/",
    x_col="file_path",
    y_col="breed",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="sparse",  # 'categorical' yerine 'sparse'
    target_size=(224, 224)
)

valid_generator = datagen.flow_from_dataframe(
    valid_data,
    directory="dog-breed-identification/train/",
    x_col="file_path",
    y_col="breed",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="sparse",  # 'categorical' yerine 'sparse'
    target_size=(224, 224)
)

Found 6542 validated image filenames belonging to 120 classes.
Found 409 validated image filenames belonging to 120 classes.


In [9]:
# Giriş boyutları (örneğin, 224x224 piksel ve 3 renk kanalı için)
input_shape = (224, 224, 3)

# Input katmanını oluştur
input_layer = Input(shape=input_shape)

# Base model oluşturma
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_layer)

# Top model oluşturma
x = base_model.output

# Giriş boyutlarına uygun olarak Conv2D ve MaxPooling2D katmanları ekleyin
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

# Geri kalan katmanlar
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

# Output katmanı
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)


In [ ]:
# Modeli derleme, loss fonksiyonu 'sparse_categorical_crossentropy' olarak değiştirildi
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early Stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modeli eğitme
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=valid_generator,
    callbacks=[early_stop]
)

Epoch 1/50


132/205 [==================>...........] - ETA: 2:42 - loss: 16.9598 - accuracy: 0.0078